In [1]:
import requests
from bs4 import BeautifulSoup as bs

url = "https://tw.stock.yahoo.com/quote/6770.TW"

response = requests.get(url)
page_html = bs(response.text, "lxml")
print(page_html)

<!DOCTYPE html>
<html class="NoJs desktop" id="atomic" lang="zh-Hant-TW"><head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charset="utf-8"/><meta content="website" property="og:type"/><meta content="股市" property="al:ios:app_name"/><meta content="790214428" property="al:ios:app_store_id"/><meta content="股市" property="al:android:app_name"/><meta content="com.yahoo.mobile.client.android.TWStock" property="al:android:package"/><meta content="ytwstock://quote/6770.TW" property="al:ios:url"/><meta content="ytwstock://quote/6770.TW" property="al:android:url"/><meta content="app-id=790214428,app-argument=ytwstock://quote/6770.TW" property="apple-itunes-app"/><meta content="力積電(6770.TW)，Yahoo奇摩股市提供您即時報價、個股走勢、成交資訊、當日籌碼，價量變化、個股相關新聞等即時資訊。" property="og:description"/><meta content="https://s.yimg.com/cv/apiv2/social/images/yahoo_default_logo-1200x1200.png" property="og:image"/><meta content="Yahoo奇摩股市" p

In [2]:
stock_name = page_html.find_all(class_="C($c-link-text) Fw(b) Fz(24px) Mend(8px) Whs(nw)")
# print(stock_name)
print(stock_name[0].get_text())

stock_price = page_html.find_all(class_="Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-down)")
print(stock_price[0].get_text())

# 用 CSS selector 篩選這種特殊的 class
# 每一個空格都是一個 class，所以每個 class 都要加"."
# 再來特殊字元要跳脫
stock_name = page_html.select(".C\\(\\$c-link-text\\).Fw\\(b\\).Fz\\(24px\\).Mend\\(8px\\).Whs\\(nw\\)")
# print(stock_name)
print(stock_name[0].get_text())


力積電
34.15
力積電


In [5]:
# 漲跌判斷
up_down_maping = {
    "border-color:transparent transparent #ff333a transparent;border-width:0 6.5px 9px 6.5px": "漲",
    "border-color:#00ab5e transparent transparent transparent;border-width:9px 6.5px 0 6.5px": "跌"
}

up_or_down_span = page_html.find("span", class_="Mend(4px) Bds(s)")
print(up_or_down_span)
up_or_down_style = up_or_down_span["style"]
print(up_or_down_style)
up_or_down = up_down_maping.get(up_or_down_style, "未知")
# up_or_down = up_down_maping.get[up_or_down_style] # 如果確定不會有不同的 style 才這樣寫
print(up_or_down)

<span class="Mend(4px) Bds(s)" style="border-color:#00ab5e transparent transparent transparent;border-width:9px 6.5px 0 6.5px"></span>
border-color:#00ab5e transparent transparent transparent;border-width:9px 6.5px 0 6.5px
跌


### 以下是延伸的其他判斷漲跌法

In [ ]:
# 嘗試使用正則判斷漲跌
import re
up_or_down_span = page_html.find("span", class_="Mend(4px) Bds(s)")
up_regex = r"#ff333a"
down_regex = r"#00ab5e"

up = re.search(up_regex, str(up_or_down_span))
down = re.search(down_regex, str(up_or_down_span))
# print(up,down) # None <re.Match object; span=(51, 58), match='#00ab5e'>

if up != None:
    print("漲")
elif down != None:
    print("跌")

None <re.Match object; span=(51, 58), match='#00ab5e'>
跌


In [19]:
# 正則更好的用法
import re
up_or_down_span = page_html.find("span", class_="Mend(4px) Bds(s)")
regex = r"(#ff333a)|(#00ab5e)"

match = re.search(regex, str(up_or_down_span))
# print(match) # <re.Match object; span=(51, 58), match='#00ab5e'>
# print(match.group(1), match.group(2)) # None #00ab5e

if match:
    if match.group(1):
        print("漲")
    else:
        print("跌")

跌


In [ ]:
# 其實可以用字串方法就好，最簡單
up_or_down_span = page_html.find("span", class_="Mend(4px) Bds(s)")

if "#ff333a" in str(up_or_down_span):
    print("漲")
elif "#00ab5e" in str(up_or_down_span):
    print("跌")

跌


In [ ]:
# 承上，但這邊考慮到可擴充性，可以這樣寫，或是當判斷條件大於兩個，這樣寫更簡潔
up_or_down_span = page_html.find("span", class_="Mend(4px) Bds(s)")

colors = {
    "#ff333a": "漲",
    "#00ab5e": "跌",
}

for key, value in colors.items():
    if key in str(up_or_down_span):
        print(value)

跌


### 打包程式碼

In [23]:
# 將程式打包成 function
import requests
from bs4 import BeautifulSoup as bs

def get_stock_info(code):
    url = f"https://tw.stock.yahoo.com/quote/{code}.TW"
    response = requests.get(url)
    page_html = bs(response.text, "lxml")

    stock_name_element = page_html.find("h1", class_="C($c-link-text) Fw(b) Fz(24px) Mend(8px) Whs(nw)")
    stock_name = stock_name_element.text

    price_parent_element = page_html.find("div", class_="D(f) Ai(fe) Mb(4px)")
    price = price_parent_element.select_one(".Fz\\(32px\\)").text

    up_down_maping = {
    "border-color:transparent transparent #ff333a transparent;border-width:0 6.5px 9px 6.5px": "漲",
    "border-color:#00ab5e transparent transparent transparent;border-width:9px 6.5px 0 6.5px": "跌"
    }
    up_or_down_style = page_html.find("span", class_="Mend(4px) Bds(s)")["style"]
    up_or_down = up_down_maping.get(up_or_down_style, "未知")

    # percent_element = price_parent_element
    percent = price_parent_element.select_one(".Jc\\(fe\\).Fz\\(20px\\)").text

    print(f"{stock_name}: {price} {up_or_down} {percent}")

In [24]:
# 使用 function
stock_codes = ["2330", "2317", "0050", "1303"]
for stock_code in stock_codes:
    get_stock_info(stock_code)

台積電: 1,430 跌 (2.05%)
鴻海: 241.0 跌 (4.37%)
元大台灣50: 61.70 跌 (1.91%)
南亞: 55.6 漲 (3.35%)
